In [ ]:
%cd /content/drive/MyDrive/Malaria

/content/drive/MyDrive/Malaria


In [ ]:
!ls

cropped_cells_log.csv	      difficult_cropped_cells_log.csv  test_data.csv
cropped_difficult_cells       infected_cropped_cells_log.csv   train_data.csv
cropped_infected_cells_train  malaria


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_train=pd.read_csv('train_data.csv')
df_test=pd.read_csv('test_data.csv')

In [ ]:
df_train.sample(5)

,ImagePAth,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center,img_width,img_height,channels,pixels
48671,images/03359d4d-69cb-4146-8286-ca7399366f97.png,red blood cell,118.0,639.0,236.0,767.0,118.0,128.0,177.0,703.0,1600,1200,3,5760000
6372,images/f06ba705-5509-479b-8067-127ffb37c5b6.png,red blood cell,535.0,141.0,659.0,234.0,124.0,93.0,597.0,187.5,1600,1200,3,5760000
21102,images/fb05b297-180a-4171-b633-09d5616d7534.png,red blood cell,977.0,151.0,1089.0,250.0,112.0,99.0,1033.0,200.5,1600,1200,3,5760000
50060,images/b1145408-feae-4637-b4cf-3e70b99cc62d.png,red blood cell,639.0,693.0,732.0,793.0,93.0,100.0,685.5,743.0,1600,1200,3,5760000
22920,images/ecd68ba2-48af-4866-826a-7cd1c147c055.png,red blood cell,1089.0,394.0,1196.0,525.0,107.0,131.0,1142.5,459.5,1600,1200,3,5760000


In [ ]:
df_train["category"].value_counts()

,count
category,
red blood cell,77420
trophozoite,1473
difficult,441
ring,353
schizont,179
gametocyte,144
leukocyte,103


In [ ]:
df_test["category"].value_counts()

,count
category,
red blood cell,5614
ring,169
trophozoite,111
gametocyte,12
schizont,11
difficult,5


Extracting Individual Infected Cells

In [ ]:
from PIL import Image
import os

In [ ]:
output_dir = "leukocyte_Cells"
os.makedirs(output_dir, exist_ok=True)

log_csv_path = "leukocyte_cells_log.csv"
non_infected_categories = ['red blood cell', 'trophozoite']
infected_categories = ['difficult', 'ring', 'schizont', 'gametocyte', 'leukocyte']

In [ ]:
selected_categories=['leukocyte']

In [ ]:
df_train = df_train.sample(frac = 1)

In [ ]:
log_data = []
count=0

for idx, row in df_train.iterrows():
    category = row['category']

    if category not in selected_categories:
        continue
    if count==1000:
        break

    image_path = "malaria/"+row['ImagePAth']
    x_min = int(row['x_min'])
    y_min = int(row['y_min'])
    x_max = int(row['x_max'])
    y_max = int(row['y_max'])

    if not os.path.exists(image_path):
        print(f"Warning: Image file {image_path} not found.")
        continue

    try:
        with Image.open(image_path) as img:
            cropped_img = img.crop((x_min, y_min, x_max, y_max))

            base_name = os.path.basename(image_path)
            file_name = f"{category}_{base_name}_{x_min}_{y_min}.png"
            output_path = os.path.join(output_dir, file_name)

            cropped_img.save(output_path)

            log_data.append({
                'original_image': image_path,
                'category': category,
                'cropped_image': output_path,
                'x_min': x_min,
                'y_min': y_min,
                'x_max': x_max,
                'y_max': y_max
            })
            count+=1

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

In [ ]:
log_df = pd.DataFrame(log_data)
log_df.to_csv(log_csv_path, index=False)

In [ ]:
log_df.shape

(103, 7)

In [ ]:
log_df.head()

,original_image,category,cropped_image,x_min,y_min,x_max,y_max
0,malaria/images/550cf4ce-b641-44a3-8487-3e16d40...,leukocyte,leukocyte_Cells/leukocyte_550cf4ce-b641-44a3-8...,902,841,997,937
1,malaria/images/f274b784-b995-4a7e-a2c2-6cc31c2...,leukocyte,leukocyte_Cells/leukocyte_f274b784-b995-4a7e-a...,61,27,154,113
2,malaria/images/9eb66577-7857-43dc-a879-8f2c9d7...,leukocyte,leukocyte_Cells/leukocyte_9eb66577-7857-43dc-a...,584,586,698,695
3,malaria/images/592c941a-aa54-4971-b499-dc47356...,leukocyte,leukocyte_Cells/leukocyte_592c941a-aa54-4971-b...,1305,986,1555,1140
4,malaria/images/4cccfcf0-cac9-46f0-870c-0eabfdc...,leukocyte,leukocyte_Cells/leukocyte_4cccfcf0-cac9-46f0-8...,1470,418,1558,526


In [ ]:
difficult_dir = "cropped_difficult_cells"
os.makedirs(difficult_dir, exist_ok=True)

In [ ]:
import shutil

In [ ]:
for idx, row in log_df.iterrows():
    if row['category'] == 'difficult':
        cropped_image_path = row['cropped_image']

        if os.path.exists(cropped_image_path):
            try:
                new_path = os.path.join(difficult_dir, os.path.basename(cropped_image_path))
                shutil.move(cropped_image_path, new_path)
                print(f"Moved {cropped_image_path} to {new_path}")
            except Exception as e:

In [ ]:
log_df[log_df['category'] != 'difficult'].to_csv('infected_cropped_cells_log.csv', index=False)

In [ ]:
log_df[log_df['category'] == 'difficult'].to_csv('difficult_cropped_cells_log.csv', index=False)